# North Carolina Hurricane Helene Analysis

This notebook analyzes water area changes in North Carolina before and after Hurricane Helene (September 27, 2024).

## Analysis Overview:
- Hurricane Helene date: September 27, 2024
- Before period: June 27, 2024 - September 27, 2024 (3 months before)
- After period: September 27, 2024 - December 27, 2024 (3 months after)
- Optional: 10-year historical analysis (2014-2024)
- Water detection using NDWI and MNDWI (Normalized Difference Water Index)
- EPSG Projection: EPSG:32617 (UTM Zone 17N for North Carolina)

## Install Required Packages (if needed)

In [ ]:
# !pip install pystac_client odc.stac folium matplotlib mapclassify geopandas

## Import Required Libraries

In [ ]:
import dask.distributed
import folium
import geopandas as gpd
import numpy as np
import shapely.geometry
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display
from pystac_client import Client
import xarray as xr

from odc.stac import configure_rio, stac_load

def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1,lon1),(lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

## Start Dask Client

This step is optional but improves load speed significantly.

In [ ]:
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
display(client)

## Define North Carolina Area of Interest

We'll focus on western North Carolina where Hurricane Helene caused significant flooding.

In [ ]:
# Define North Carolina bounding box
# Focusing on Asheville and western NC area (heavily impacted by Hurricane Helene)
# Coordinates: longitude, latitude
nc_west_center = (-82.5515, 35.5951)  # Asheville, NC area

# Create bounding box (approximately 100km x 100km)
km2deg = 1.0 / 111
x, y = nc_west_center
r = 50 * km2deg  # 50 km radius

bbox = (x - r, y - r, x + r, y + r)
print(f"Bounding box: {bbox}")

# Hurricane Helene date
hurricane_date = datetime(2024, 9, 27)
print(f"Hurricane Helene date: {hurricane_date.strftime('%Y-%m-%d')}")

## Query Sentinel-2 Data: 3 Months Before Hurricane Helene

In [ ]:
# Define date ranges
before_start = (hurricane_date - timedelta(days=90)).strftime('%Y-%m-%d')
before_end = hurricane_date.strftime('%Y-%m-%d')

print(f"Before period: {before_start} to {before_end}")

# Connect to STAC catalog
catalog = Client.open("https://earth-search.aws.element84.com/v1/")

# Query for data before the hurricane
query_before = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime=f"{before_start}/{before_end}",
    bbox=bbox,
    limit=100,
    query={"eo:cloud_cover": {"lt": 20}}  # Less than 20% cloud cover
)

items_before = list(query_before.items())
print(f"Found {len(items_before)} datasets before the hurricane")

# Convert to GeoJSON
stac_json_before = query_before.item_collection_as_dict()

## Query Sentinel-2 Data: 3 Months After Hurricane Helene

In [ ]:
# Define date ranges
after_start = hurricane_date.strftime('%Y-%m-%d')
after_end = (hurricane_date + timedelta(days=90)).strftime('%Y-%m-%d')

print(f"After period: {after_start} to {after_end}")

# Query for data after the hurricane
query_after = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime=f"{after_start}/{after_end}",
    bbox=bbox,
    limit=100,
    query={"eo:cloud_cover": {"lt": 20}}  # Less than 20% cloud cover
)

items_after = list(query_after.items())
print(f"Found {len(items_after)} datasets after the hurricane")

# Convert to GeoJSON
stac_json_after = query_after.item_collection_as_dict()

## Visualize Query Results

In [ ]:
# Create GeoDataFrames
if len(items_before) > 0:
    gdf_before = gpd.GeoDataFrame.from_features(stac_json_before, "epsg:4326")
    print(f"\nBefore hurricane - {len(gdf_before)} granules")
    display(gdf_before[['datetime', 'eo:cloud_cover']].head())

if len(items_after) > 0:
    gdf_after = gpd.GeoDataFrame.from_features(stac_json_after, "epsg:4326")
    print(f"\nAfter hurricane - {len(gdf_after)} granules")
    display(gdf_after[['datetime', 'eo:cloud_cover']].head())

## Load Data with North Carolina EPSG:32617 Projection

We use EPSG:32617 (UTM Zone 17N) which is appropriate for North Carolina.

In [ ]:
# Load data before hurricane
if len(items_before) > 0:
    xx_before = stac_load(
        items_before,
        bands=["red", "green", "blue", "nir", "swir16"],  # Include NIR and SWIR for water detection
        crs="EPSG:32617",  # UTM Zone 17N for North Carolina
        resolution=100,  # 100m resolution
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day",
        bbox=bbox,
    )
    print(f"\nBefore hurricane data shape: {xx_before.dims}")
    print(xx_before)

In [ ]:
# Load data after hurricane
if len(items_after) > 0:
    xx_after = stac_load(
        items_after,
        bands=["red", "green", "blue", "nir", "swir16"],
        crs="EPSG:32617",  # UTM Zone 17N for North Carolina
        resolution=100,  # 100m resolution
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day",
        bbox=bbox,
    )
    print(f"\nAfter hurricane data shape: {xx_after.dims}")
    print(xx_after)

## Water Detection Function

We'll use NDWI (Normalized Difference Water Index) to detect water areas.
NDWI = (Green - NIR) / (Green + NIR)

We'll also use MNDWI (Modified NDWI) for better water detection:
MNDWI = (Green - SWIR) / (Green + SWIR)

In [ ]:
def calculate_ndwi(data):
    """
    Calculate NDWI (Normalized Difference Water Index)
    NDWI = (Green - NIR) / (Green + NIR)
    
    Water typically has NDWI > 0
    """
    green = data['green'].astype('float32')
    nir = data['nir'].astype('float32')
    
    ndwi = (green - nir) / (green + nir + 1e-10)  # Add small value to avoid division by zero
    return ndwi

def calculate_mndwi(data):
    """
    Calculate MNDWI (Modified Normalized Difference Water Index)
    MNDWI = (Green - SWIR) / (Green + SWIR)
    
    Better for distinguishing water from built-up areas
    Water typically has MNDWI > 0
    """
    green = data['green'].astype('float32')
    swir = data['swir16'].astype('float32')
    
    mndwi = (green - swir) / (green + swir + 1e-10)
    return mndwi

def create_water_mask(data, threshold=0.0, use_mndwi=True):
    """
    Create a binary water mask
    
    Args:
        data: xarray dataset with required bands
        threshold: threshold value for water detection (default 0.0)
        use_mndwi: if True, use MNDWI; otherwise use NDWI
    
    Returns:
        Binary mask where 1 = water, 0 = non-water
    """
    if use_mndwi:
        index = calculate_mndwi(data)
    else:
        index = calculate_ndwi(data)
    
    water_mask = (index > threshold).astype(int)
    return water_mask, index

print("Water detection functions defined.")

## Compute and Visualize Water Masks - Before Hurricane

In [ ]:
if len(items_before) > 0:
    # Compute data
    print("Loading before hurricane data...")
    xx_before_computed = xx_before.compute()
    
    # Create water mask for the most recent date before hurricane
    if len(xx_before_computed.time) > 0:
        latest_before = xx_before_computed.isel(time=-1)  # Get most recent
        water_mask_before, mndwi_before = create_water_mask(latest_before, use_mndwi=True)
        
        print(f"\nBefore hurricane - Date: {latest_before.time.values}")
        print(f"Water pixels detected: {water_mask_before.sum().values:,}")
        print(f"Total pixels: {water_mask_before.size:,}")
        print(f"Water percentage: {(water_mask_before.sum() / water_mask_before.size * 100).values:.2f}%")

In [ ]:
if len(items_before) > 0 and len(xx_before_computed.time) > 0:
    # Visualize RGB and water mask
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # RGB composite
    rgb = np.stack([
        latest_before['red'].values,
        latest_before['green'].values,
        latest_before['blue'].values
    ], axis=-1)
    rgb = np.clip(rgb / 3000, 0, 1)  # Normalize to 0-1
    
    axes[0].imshow(rgb)
    axes[0].set_title('RGB Composite - Before Hurricane')
    axes[0].axis('off')
    
    # MNDWI
    im1 = axes[1].imshow(mndwi_before, cmap='RdYlBu', vmin=-0.5, vmax=0.5)
    axes[1].set_title('MNDWI - Before Hurricane')
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Water mask
    im2 = axes[2].imshow(water_mask_before, cmap='Blues', vmin=0, vmax=1)
    axes[2].set_title('Water Mask - Before Hurricane')
    axes[2].axis('off')
    plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

## Compute and Visualize Water Masks - After Hurricane

In [ ]:
if len(items_after) > 0:
    # Compute data
    print("Loading after hurricane data...")
    xx_after_computed = xx_after.compute()
    
    # Create water mask for the earliest date after hurricane
    if len(xx_after_computed.time) > 0:
        earliest_after = xx_after_computed.isel(time=0)  # Get earliest after
        water_mask_after, mndwi_after = create_water_mask(earliest_after, use_mndwi=True)
        
        print(f"\nAfter hurricane - Date: {earliest_after.time.values}")
        print(f"Water pixels detected: {water_mask_after.sum().values:,}")
        print(f"Total pixels: {water_mask_after.size:,}")
        print(f"Water percentage: {(water_mask_after.sum() / water_mask_after.size * 100).values:.2f}%")

In [ ]:
if len(items_after) > 0 and len(xx_after_computed.time) > 0:
    # Visualize RGB and water mask
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # RGB composite
    rgb = np.stack([
        earliest_after['red'].values,
        earliest_after['green'].values,
        earliest_after['blue'].values
    ], axis=-1)
    rgb = np.clip(rgb / 3000, 0, 1)  # Normalize to 0-1
    
    axes[0].imshow(rgb)
    axes[0].set_title('RGB Composite - After Hurricane')
    axes[0].axis('off')
    
    # MNDWI
    im1 = axes[1].imshow(mndwi_after, cmap='RdYlBu', vmin=-0.5, vmax=0.5)
    axes[1].set_title('MNDWI - After Hurricane')
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Water mask
    im2 = axes[2].imshow(water_mask_after, cmap='Blues', vmin=0, vmax=1)
    axes[2].set_title('Water Mask - After Hurricane')
    axes[2].axis('off')
    plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

## Compare Water Areas Before and After Hurricane

In [ ]:
if (len(items_before) > 0 and len(items_after) > 0 and 
    len(xx_before_computed.time) > 0 and len(xx_after_computed.time) > 0):
    
    # Calculate change in water extent
    water_change = water_mask_after.astype(float) - water_mask_before.astype(float)
    
    # Statistics
    new_water = (water_change > 0).sum().values
    lost_water = (water_change < 0).sum().values
    unchanged = (water_change == 0).sum().values
    
    print("\n=== Water Area Change Analysis ===")
    print(f"New water areas (flooding): {new_water:,} pixels")
    print(f"Reduced water areas: {lost_water:,} pixels")
    print(f"Unchanged areas: {unchanged:,} pixels")
    print(f"Net change: {new_water - lost_water:,} pixels")
    
    # Visualize change
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    im = ax.imshow(water_change, cmap='RdBu_r', vmin=-1, vmax=1)
    ax.set_title('Water Area Change: Before vs After Hurricane Helene\n(Blue = New Water/Flooding, Red = Water Loss)')
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label='Change')
    plt.tight_layout()
    plt.show()

## 10-Year Historical Analysis (Optional)

This section allows for querying and analyzing water extent over a 10-year period (2014-2024).

In [ ]:
def query_historical_data(bbox, start_year, end_year, month=9):
    """
    Query historical Sentinel-2 data for multiple years.
    
    Args:
        bbox: Bounding box
        start_year: Start year for analysis
        end_year: End year for analysis
        month: Month to focus on (default 9 for September, hurricane season)
    
    Returns:
        Dictionary with years as keys and STAC items as values
    """
    catalog = Client.open("https://earth-search.aws.element84.com/v1/")
    historical_data = {}
    
    # Note: Sentinel-2 data is available from 2015 onwards
    actual_start_year = max(start_year, 2015)
    
    for year in range(actual_start_year, end_year + 1):
        # Query for the same time period each year
        date_start = f"{year}-{month:02d}-01"
        date_end = f"{year}-{month:02d}-30"
        
        print(f"Querying year {year}: {date_start} to {date_end}")
        
        query = catalog.search(
            collections=["sentinel-2-l2a"],
            datetime=f"{date_start}/{date_end}",
            bbox=bbox,
            limit=10,
            query={"eo:cloud_cover": {"lt": 20}}
        )
        
        items = list(query.items())
        historical_data[year] = items
        print(f"  Found {len(items)} datasets")
    
    return historical_data

print("Historical analysis function defined.")
print("\nTo run 10-year analysis, execute the next cell.")
print("Note: This will take significant time to download and process data.")

In [ ]:
# Uncomment to run 10-year historical analysis
# WARNING: This will take considerable time and resources

# historical_data = query_historical_data(bbox, start_year=2014, end_year=2024, month=9)

# # Process each year
# historical_water_stats = {}
# for year, items in historical_data.items():
#     if len(items) > 0:
#         # Load data for this year
#         xx_year = stac_load(
#             items,
#             bands=["green", "nir", "swir16"],
#             crs="EPSG:32617",
#             resolution=100,
#             chunks={"x": 2048, "y": 2048},
#             groupby="solar_day",
#             bbox=bbox,
#         )
#         
#         xx_year_computed = xx_year.compute()
#         
#         if len(xx_year_computed.time) > 0:
#             # Use median composite for the month
#             median_data = xx_year_computed.median(dim='time')
#             water_mask_year, _ = create_water_mask(median_data, use_mndwi=True)
#             
#             water_pct = (water_mask_year.sum() / water_mask_year.size * 100).values
#             historical_water_stats[year] = water_pct
#             print(f"Year {year}: {water_pct:.2f}% water coverage")

# # Plot historical trend
# if historical_water_stats:
#     years = list(historical_water_stats.keys())
#     water_pcts = list(historical_water_stats.values())
#     
#     plt.figure(figsize=(12, 6))
#     plt.plot(years, water_pcts, marker='o', linewidth=2, markersize=8)
#     plt.axvline(x=2024, color='red', linestyle='--', label='Hurricane Helene (2024)')
#     plt.xlabel('Year', fontsize=12)
#     plt.ylabel('Water Coverage (%)', fontsize=12)
#     plt.title('10-Year Water Coverage Trend - North Carolina (September)', fontsize=14)
#     plt.grid(True, alpha=0.3)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

## Summary and Export

This notebook provides tools to:
1. Query Sentinel-2 imagery for North Carolina before and after Hurricane Helene
2. Use EPSG:32617 (UTM Zone 17N) projection for North Carolina
3. Detect water areas using NDWI and MNDWI indices
4. Compare water extent before and after the hurricane
5. Optionally analyze 10-year historical trends

### Key Parameters:
- **EPSG Code**: 32617 (UTM Zone 17N)
- **Water Detection**: MNDWI (Modified Normalized Difference Water Index)
- **Hurricane Date**: September 27, 2024
- **Analysis Period**: 3 months before and after
- **Study Area**: Western North Carolina (Asheville region)

In [ ]:
# Export water masks to GeoTIFF (optional)
# Uncomment to save results

# if len(items_before) > 0 and len(xx_before_computed.time) > 0:
#     water_mask_before.rio.to_raster("../outputs/water_mask_before_hurricane.tif")
#     print("Saved: water_mask_before_hurricane.tif")

# if len(items_after) > 0 and len(xx_after_computed.time) > 0:
#     water_mask_after.rio.to_raster("../outputs/water_mask_after_hurricane.tif")
#     print("Saved: water_mask_after_hurricane.tif")

# if (len(items_before) > 0 and len(items_after) > 0 and 
#     len(xx_before_computed.time) > 0 and len(xx_after_computed.time) > 0):
#     water_change.rio.to_raster("../outputs/water_change_hurricane_helene.tif")
#     print("Saved: water_change_hurricane_helene.tif")